In [3]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = r'C:\Users\Dell\Desktop\Project_2\Airbnb_Open_Data1.csv\Airbnb_Open_Data.csv'  # Update with your file path
df = pd.read_csv(file_path)

# Replace misspelled names with correct ones
df['neighbourhood'] = df['neighbourhood'].replace({'brookln': 'Brooklyn', 'manhatan': 'Manhattan'})

# 1. Remove duplicates where all columns for a specific row contain the same information
df.drop_duplicates(inplace=True)

# 2. Remove additional spaces and enters between words in text fields
df = df.applymap(lambda x: ' '.join(str(x).split()) if isinstance(x, str) else x)

# 3. Change uppercase words in column titles to lowercase and replace spaces with underscores
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

# 4. Remove '$' character from 'price' and 'service_fee' columns and convert to float
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
df['service_fee'] = df['service_fee'].replace('[\$,]', '', regex=True).astype(float)

# Fill empty cells for object and numeric types with appropriate values
object_columns = df.select_dtypes(include=['object']).columns
for col in object_columns:
    df[col] = df[col].fillna('Unspecified').astype(str)

# Fill empty cells for numeric types with NaN
for col in df.select_dtypes(include=['number']).columns:
    df[col].fillna(pd.NA, inplace=True)

# 6. Delete 'license', 'country_code', and 'county' columns if they exist
columns_to_remove = ['license', 'country_code', 'county']
df.drop(columns=[col for col in columns_to_remove if col in df.columns], inplace=True)

# 10. If the column contains text, capitalize the first letter
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.capitalize()
    
# Convert appropriate columns to numerical or datetime types
def safe_convert_to_float(x):
    try:
        return float(x)
    except ValueError:
        return np.nan

df['lat'] = df['lat'].apply(safe_convert_to_float)
df['long'] = df['long'].apply(safe_convert_to_float)
df['construction_year'] = pd.to_numeric(df['construction_year'], errors='coerce')
df['minimum_nights'] = pd.to_numeric(df['minimum_nights'], errors='coerce')
df['number_of_reviews'] = pd.to_numeric(df['number_of_reviews'], errors='coerce')
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')
df['reviews_per_month'] = pd.to_numeric(df['reviews_per_month'], errors='coerce')
df['review_rate_number'] = pd.to_numeric(df['review_rate_number'], errors='coerce')
df['calculated_host_listings_count'] = pd.to_numeric(df['calculated_host_listings_count'], errors='coerce')
df['availability_365'] = pd.to_numeric(df['availability_365'], errors='coerce')

# 8. Provide the types of each column for importing the data to pgadmin
column_types = df.dtypes
print("Column types for pgadmin import:")
print(column_types)

# 9. Identify any other issue for the import and eliminate it
# Check for any columns with mixed types
for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, (str, bool, np.number))).all() == False:
        print(f"Column {col} contains mixed types and needs to be standardized.")

        # Save the cleaned data to a new CSV file
output_file_path = r'C:\Users\Dell\Desktop\Project_2\Airbnb_Open_Data1.csv\Cleaned_Airbnb_Open_Data.csv'  # Use raw string to avoid unicode errors
df.to_csv(output_file_path, index=False)

print("Cleaned data has been saved to:", output_file_path)

C:\Users\Dell\AppData\Local\Temp\ipykernel_25112\3603171623.py:6: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Column types for pgadmin import:
id                                         int64
name                                      object
host_id                                    int64
host_identity_verified                    object
host_name                                 object
neighbourhood_group                       object
neighbourhood                             object
lat                                      float64
long                                     float64
country                                   object
instant_bookable                          object
cancellation_policy                       object
room_type                                 object
construction_year                        float64
price                                    float64
service_fee                              float64
minimum_nights                           float64
number_of_reviews                        float64
last_review                       datetime64[ns]
reviews_per_month                   

In [76]:
df.head(10)

,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,country,...,price,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules
0,1001254,Clean & quiet apt home by the park,80014485718,Unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United states,...,966.0,193.0,10.0,9.0,2021-10-19,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...
1,1002102,Skylit midtown castle,52335172823,Verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United states,...,142.0,28.0,30.0,45.0,2022-05-21,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...
2,1002403,The village of harlem....new york !,78829239556,Unspecified,Elise,Manhattan,Harlem,40.80902,-73.94190,United states,...,620.0,124.0,3.0,0.0,NaT,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and..."
3,1002755,Unspecified,85098326012,Unconfirmed,Garry,Brooklyn,Clinton hill,40.68514,-73.95976,United states,...,368.0,74.0,30.0,270.0,2019-07-05,4.64,4.0,1.0,322.0,Unspecified
4,1003689,Entire apt: spacious studio/loft by central park,92037596077,Verified,Lyndon,Manhattan,East harlem,40.79851,-73.94399,United states,...,204.0,41.0,10.0,9.0,2018-11-19,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th..."
5,1004098,Large cozy 1 br apartment in midtown east,45498551794,Verified,Michelle,Manhattan,Murray hill,40.74767,-73.97500,United states,...,577.0,115.0,3.0,74.0,2019-06-22,0.59,3.0,1.0,374.0,"No smoking, please, and no drugs."
6,1004650,Blissartsspace!,61300605564,Unspecified,Alberta,Brooklyn,Bedford-stuyvesant,40.68688,-73.95596,United states,...,71.0,14.0,45.0,49.0,2017-10-05,0.40,5.0,1.0,224.0,Please no shoes in the house so bring slippers...
7,1005202,Blissartsspace!,90821839709,Unconfirmed,Emma,Brooklyn,Bedford-stuyvesant,40.68688,-73.95596,United states,...,1060.0,212.0,45.0,49.0,2017-10-05,0.40,5.0,1.0,219.0,House guidelines for our bnb we are delighted ...
8,1005754,Large furnished room near b'way,79384379533,Verified,Evelyn,Manhattan,Hell's kitchen,40.76489,-73.98493,United states,...,1018.0,204.0,2.0,430.0,2019-06-24,3.47,3.0,1.0,180.0,- please clean up after yourself when using th...
9,1006307,Cozy clean guest room - family apt,75527839483,Unconfirmed,Carl,Manhattan,Upper west side,40.80178,-73.96723,United states,...,291.0,58.0,2.0,118.0,2017-07-21,0.99,5.0,1.0,375.0,No smoking or pets anywhere on the property 1....


In [2]:
# Save the cleaned data to a new CSV file
output_file_path = r'C:\Users\Dell\Desktop\Project_2\Airbnb_Open_Data1.csv\Cleaned_Airbnb_Open_Data.csv'  # Use raw string to avoid unicode errors
df.to_csv(output_file_path, index=False)

print("Cleaned data has been saved to:", output_file_path)

Cleaned data has been saved to: C:\Users\Dell\Desktop\Project_2\Airbnb_Open_Data1.csv\Cleaned_Airbnb_Open_Data.csv


In [79]:
df['instant_bookable']

0         False
1         False
2          True
3          True
4         False
          ...  
102053     True
102054    False
102055     True
102056     True
102057    False
Name: instant_bookable, Length: 102058, dtype: object